In [44]:
import numpy as np
import sys
from six import StringIO, b

from gym import utils
from gym.envs.toy_text import discrete

LEFT = 0
DOWN = 1
RIGHT = 2
UP = 3

MAPS = {
    "4x4": [
        "SFFF",
        "FHFH",
        "FFFH",
        "HFFG"
    ],
    "8x8": [
        "SFFFFFFF",
        "FFFFFFFF",
        "FFFHFFFF",
        "FFFFFHFF",
        "FFFHFFFF",
        "FHHFFFHF",
        "FHFFHFHF",
        "FFFHFFFG"
    ],
}

class FrozenLakeEnv(discrete.DiscreteEnv):
    """
    Winter is here. You and your friends were tossing around a frisbee at the park
    when you made a wild throw that left the frisbee out in the middle of the lake.
    The water is mostly frozen, but there are a few holes where the ice has melted.
    If you step into one of those holes, you'll fall into the freezing water.
    At this time, there's an international frisbee shortage, so it's absolutely imperative that
    you navigate across the lake and retrieve the disc.
    However, the ice is slippery, so you won't always move in the direction you intend.
    The surface is described using a grid like the following
        SFFF
        FHFH
        FFFH
        HFFG
    S : starting point, safe
    F : frozen surface, safe
    H : hole, fall to your doom
    G : goal, where the frisbee is located
    The episode ends when you reach the goal or fall in a hole.
    You receive a reward of 1 if you reach the goal, and zero otherwise.
    """

    metadata = {'render.modes': ['human', 'ansi']}

    def __init__(self, desc=None, map_name="4x4",is_slippery=True):
        if desc is None and map_name is None:
            raise ValueError('Must provide either desc or map_name')
        elif desc is None:
            desc = MAPS[map_name]
        self.desc = desc = np.asarray(desc,dtype='c')
        self.nrow, self.ncol = nrow, ncol = desc.shape
        self.reward_range = (0, 1)

        nA = 4
        nS = nrow * ncol

        isd = np.array(desc == b'S').astype('float64').ravel()
        isd /= isd.sum()

        P = {s : {a : [] for a in range(nA)} for s in range(nS)}

        def to_s(row, col):
            return row*ncol + col
        
        def inc(row, col, a):
            if a==0: # left
                col = max(col-1,0)
            elif a==1: # down
                row = min(row+1,nrow-1)
            elif a==2: # right
                col = min(col+1,ncol-1)
            elif a==3: # up
                row = max(row-1,0)
            return (row, col)

        for row in range(nrow):
            for col in range(ncol):
                s = to_s(row, col)
                for a in range(4):
                    li = P[s][a]
                    letter = desc[row, col]
                    if letter in b'GH':
                        li.append((1.0, s, 0, True))
                    else:
                        if is_slippery:
                            for b in [(a-1)%4, a, (a+1)%4]:
                                newrow, newcol = inc(row, col, b)
                                newstate = to_s(newrow, newcol)
                                newletter = desc[newrow, newcol]
                                done = bytes(newletter) in b'GH'
                                rew = float(newletter == b'G')
                                li.append((1.0/3.0, newstate, rew, done))
                        else:
                            newrow, newcol = inc(row, col, a)
                            newstate = to_s(newrow, newcol)
                            newletter = desc[newrow, newcol]
                            done = bytes(newletter) in b'GH'
                            rew = float(newletter == b'G')
                            li.append((1.0, newstate, rew, done))

        super(FrozenLakeEnv, self).__init__(nS, nA, P, isd)

    def render(self, mode='human'):
        outfile = StringIO() if mode == 'ansi' else sys.stdout

        row, col = self.s // self.ncol, self.s % self.ncol
        desc = self.desc.tolist()
        desc = [[c.decode('utf-8') for c in line] for line in desc]
        desc[row][col] = utils.colorize(desc[row][col], "red", highlight=True)
        if self.lastaction is not None:
            outfile.write("  ({})\n".format(["Left","Down","Right","Up"][self.lastaction]))
        else:
            outfile.write("\n")
        outfile.write("\n".join(''.join(line) for line in desc)+"\n")

        if mode != 'human':
            return outfile

In [204]:
vector = [3, 2, 3, 3]
x = np.max(vector)
indices = np.nonzero(vector == x)[0] # 인덱스 자체가 랜덤 방향이 되므로.
import random
random.choice(indices)

3

In [ ]:
import gym
import random as rand
import numpy as np

env = gym.make('FrozenLake-v0')

LEFT = 0
DOWN = 1
RIGHT = 2
UP = 3

Q = np.zeros([env.observation_space.n,env.action_space.n])
# Q[0,RIGHT] = 1
# Q[1,RIGHT] = 1
# Q[2,DOWN] = 1
# Q[6,DOWN] = 1
# Q[10,DOWN] = 1
# Q[14,RIGHT] = 1
# print(Q)

def greedy_action(vector):
    m = np.amax(vector)
    indices = np.nonzero(vector == m)[0]
    return random.choice(indices)

def random_action():
    dirs = [ LEFT, DOWN, RIGHT, UP ]
    return rand.choice(dirs)

for step in range(100):
    prev_state = env.reset()
    doomed = False
    state_list = []
    while not doomed:
        state_list.append(prev_state)
        action = greedy_action(Q[prev_state,:]) # random_action()
        curr_state, reward, doomed, _ = env.step(action) 
        if not doomed and prev_state != curr_state:
            env.render()
            print("[ING.][%d][%d->%d] : %f" % (step, prev_state, curr_state, reward))
        if doomed:
#             env.render()
#             print("[FAIL][%d][%d->%d] : %f\n" % (step, prev_state, curr_state, reward))
            sys.stdout.write(".")
        elif reward == 1:
            env.render()
            print("[SUCC][%d][%d->%d] : %f" % (step, prev_state, curr_state, reward))
#         Q[prev_state, action] = reward + np.max(Q[curr_state,:])
        prev_state = curr_state
        
        

In [261]:
#Initialize table with all zeros
Q = np.zeros([env.observation_space.n,env.action_space.n])
# Set learning parameters
lr = .8
y = .95
num_episodes = 2000
#create lists to contain total rewards and steps per episode
#jList = []
rList = []
for i in range(num_episodes):
    #Reset environment and get first new observation
    s = env.reset()
    rAll = 0
    d = False
    j = 0
    #The Q-Table learning algorithm
    while j < 99:
        j+=1
        #Choose an action by greedily (with noise) picking from Q table
        a = np.argmax(Q[s,:] + np.random.randn(1,env.action_space.n)*(1./(i+1)))
        #Get new state and reward from environment
        s1,r,d,_ = env.step(a)
        #Update Q-Table with new knowledge
        Q[s,a] = Q[s,a] + lr*(r + y*np.max(Q[s1,:]) - Q[s,a])
        rAll += r
        s = s1
        if d == True:
            break
    #jList.append(j)
    rList.append(rAll)
    

print("Final Q-Table Values")
print(Q)

Final Q-Table Values
[[  9.31567542e-02   2.37357336e-03   6.86358220e-03   4.68377009e-03]
 [  7.44819213e-04   4.17219667e-04   1.02115902e-04   1.09115004e-01]
 [  2.63770691e-01   2.90019242e-03   3.50736030e-03   3.72568029e-03]
 [  7.41257218e-04   1.80956883e-04   2.90070182e-03   5.41828667e-03]
 [  2.10120370e-01   3.52319655e-04   1.38801754e-03   7.71427253e-04]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00]
 [  1.90316645e-01   2.96620657e-07   7.70632133e-05   1.77991297e-04]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00]
 [  5.19031543e-03   3.20472359e-04   5.27009537e-03   4.71748779e-01]
 [  3.61630154e-04   2.50066266e-01   0.00000000e+00   1.23449141e-04]
 [  6.72650187e-01   6.76299892e-04   0.00000000e+00   2.04220829e-03]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00]
 [  0.00000000e+00   0.00000000e+00   4.17589624e-01   0